IMPORTANT: First install bs4 from the terminal if there is no previous : pip install beautifulsoup4


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


### Import data as dataframe


In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')
#print(soup.prettify())

match = soup.table.text

table will get the tag element together with text
If you use match method, it will match the first tag, find would be better and more specificdiv

match = soup.find('div',class ='_footer_')
match = soup.find('div', id ='footer')

In this assignment, we use find(). 

first we use find() to find the content of table and store it in one variable called table, then we use find_all() to find all content of attributes of 'td' 

In [10]:
table = soup.find('table')
fields = table.find_all('td')
print(fields[0:10])
print(len(fields))

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
</td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
</td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td>, <td>M4A</td>]
864


Some explaination of syntax:
range() function syntax range (start, stop[, step])

string.strip()

Example:
str = "0000000this is string example....wow!!!0000000";
print str.strip( '0' )

output: this is string example....wow!!!

The following code is going to create 3 lists to store Postcode, Borough and Neighbourhood respectively. And append the content in every td tag

In [11]:
postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        

The following codes will append 3 list into a dataframe and we will transpose it


In [12]:
df1 = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df1.columns = ['Postcode', 'Borough', 'Neighbourhood']
df1.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only keep assigned column and drop unassigned col

In [13]:
df1['Borough'].replace('Not assigned', np.nan, inplace=True)
df1.dropna(subset=['Borough'], inplace=True)
df1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Combine the borough with the same postcode.The combination of Postcode and Borough should be unique after groupby. Combine the values of Neighbourhood by and use ',' as delimilator


In [14]:
df2 = df1.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df2.columns = ['Postcode', 'Borough', 'Neighbourhood']
df2.shape
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
df2.shape

(103, 3)

Read geo data as a dataframe, give it column names 


In [16]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')
geo.columns = ['Postcode', 'Latitude', 'Longitude']
geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


merge this dataframe with df2 on column Postcode. 

In [17]:
dfFinal = pd.merge(df2, geo, on=['Postcode'], how='inner')
dfFinal.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Create a map of Tornoto

In [18]:
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


In [19]:
# create map of toronto using latitude and longitude
mapToronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers 
for lat, lng, borough, neighborhood in zip(dfFinal['Latitude'], dfFinal['Longitude'], dfFinal['Borough'], dfFinal['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(mapToronto)  
    
mapToronto

### Explore Toronto Neighbourhoods


#### Define Foursquare Credentials and Version

In [51]:
CLIENT_ID = 'xxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxxxxx
CLIENT_SECRET:xxxxxx


#### Only keep Toronto data

In [23]:
dfFinal = dfFinal[dfFinal['Borough'].str.contains('Toronto')]
to_data = dfFinal.reset_index(drop=True)
to_data


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [24]:
# create map of Toronto using latitude and longitude values
map_tohood = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(to_data['Latitude'], to_data['Longitude'], to_data['Borough'], to_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_tohood)  
    
map_tohood

#### Explore the Neighbourhoods

In [26]:
to_data.loc[0, 'Neighbourhood']

'The Beaches'

In [27]:
neighbourhood_latitude = to_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = to_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = to_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### Find the top 10 venus are within a radius from the centroid of 5 km

In [47]:
LIMIT = 10
radius = 5000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=XBX4CP4QYLURFNZMSFO3Z3TVUWBNW1TAIZSRXDVSUJYDL2CZ&client_secret=5CZBFSS0A1ZY31Y4TB5KUP13RB5Q2B5POGIRBCMB3UTBIFQD&v=20180605&ll=43.67635739999999,-79.2930312&radius=5000&limit=10'

In [48]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cc2b7d64c1f6743903168d2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 231,
  'suggestedBounds': {'ne': {'lat': 43.72135744500004,
    'lng': -79.23092834490522},
   'sw': {'lat': 43.63135735499995, 'lng': -79.35513405509478}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a520c4f720e3',
       'name': 'The Fox Theatre',
       'location': {'address': '2236 Queen St East',
        'crossStreet': 'at Beech Ave',
        'lat': 43.67280115048827,
        'lng': -79.28727198292374,
        'labeledLatLngs': [{'label': 'display',



#### define a function to get category of the venue

In [49]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Read JSON file into a dataframe

In [50]:

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Fox Theatre,Indie Movie Theater,43.672801,-79.287272
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
2,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993
3,The Beech Tree,Gastropub,43.680493,-79.288846
4,Bagels On Fire,Bagel Shop,43.672864,-79.286784


#### The above is the top 4 venues